In [2]:
import numpy as np
import pandas as pd

from datasets import load_from_disk

In [3]:
qa_dataset = load_from_disk('/home/ddz5/Desktop/c2s-RL/hf_dataset_2024-12-06')

In [4]:
qa_dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'cell_sentences_data', 'Question', 'Answer', 'Keyword', 'full_QA_pair', 'Publication_Title', 'Dataset_URL', 'Dataset_Index', 'Used_Rows', 'openai_batch_response_id'],
        num_rows: 566
    })
    validation: Dataset({
        features: ['Context', 'cell_sentences_data', 'Question', 'Answer', 'Keyword', 'full_QA_pair', 'Publication_Title', 'Dataset_URL', 'Dataset_Index', 'Used_Rows', 'openai_batch_response_id'],
        num_rows: 105
    })
    test: Dataset({
        features: ['Context', 'cell_sentences_data', 'Question', 'Answer', 'Keyword', 'full_QA_pair', 'Publication_Title', 'Dataset_URL', 'Dataset_Index', 'Used_Rows', 'openai_batch_response_id'],
        num_rows: 99
    })
})

In [5]:
# Unique publications
len(set(qa_dataset['train']['Publication_Title'])) + len(set(qa_dataset['validation']['Publication_Title'])) + len(set(qa_dataset['test']['Publication_Title']))

26

In [6]:
# unique datasets
dataset_local_indices = set(qa_dataset['train']['Dataset_Index']) | set(qa_dataset['test']['Dataset_Index']) | set(qa_dataset['validation']['Dataset_Index'])
len(dataset_local_indices)

26

In [7]:
match_datasets = pd.read_csv('/gpfs/radev/home/ddz5/Desktop/c2s-RL/RL_data/matching_datasets.csv')

In [8]:
match_datasets

,dataset,filename
0,Activated PBMCs - Expanded cohort CITE-seq,local(304).h5ad
1,blood and bone marrow from a healthy young donor,local(477).h5ad
2,scRNA-seq analysis of healthy human fallopian ...,local(770).h5ad
3,HypoMap – a unified single cell gene expressio...,local(52).h5ad
4,L4 IT - MTG: Seattle Alzheimer's Disease Atlas...,local(274).h5ad
...,...,...
86,Molecular characterization of selectively vuln...,local(604).h5ad
87,Single-cell longitudinal analysis of SARS-CoV-...,local(778).h5ad
88,Atlas of the developing mouse thalamus,local(466).h5ad
89,Colon,local(777).h5ad


In [12]:
match_datasets['dataset'].nunique()

91

In [9]:
all_local_indices = []
for filename in match_datasets['filename']:
    all_local_indices.append(filename.split('local(')[1].split(').h5ad')[0])

In [10]:
len(all_local_indices)

91

In [11]:
num_datasets = 0
for local_indices in list(dataset_local_indices):
    if local_indices not in all_local_indices:
        num_datasets += 1
        print(local_indices)

593
503
678


In [21]:
intersection = dataset_local_indices & set(['593', '503', '678'])
union = dataset_local_indices | set(['593', '503', '678'])

used_dataset_indices = union - intersection

In [101]:
for index in used_dataset_indices:
    print(f'\"local({index})\"')

"local(706)"
"local(474)"
"local(462)"
"local(486)"
"local(738)"
"local(603)"
"local(698)"
"local(513)"
"local(52)"
"local(471)"
"local(23)"
"local(50)"
"local(546)"
"local(728)"
"local(778)"
"local(267)"
"local(635)"
"local(632)"
"local(777)"
"local(633)"
"local(473)"
"local(636)"
"local(739)"


In [28]:
import anndata as ad

In [97]:
dataset = ad.read_h5ad('/home/sz568/scratch/C2S_RL/all_datasets/hca_cellxgene_cleaned_h5ad/local(471)_cleaned.h5ad')

In [98]:
dataset

AnnData object with n_obs × n_vars = 69645 × 26163
    obs: 'cell_id', 'cell_type', 'tissue', 'donor_id', 'sample_id', 'batch_condition', 'organism', 'assay', 'disease', 'sex', 'self_reported_ethnicity', 'development_stage', 'orig_filename', 'rahul_tissue_categ'
    var: 'gene_name', 'ensembl_id', 'gene_name_w_version_num', 'ensembl_id_w_version_num'

In [49]:
dataset.var["gene_name"][-10:]

AP001055                  AP001055
DNMT3L                      DNMT3L
TRPM2-AS                  TRPM2-AS
KRTAP10-4                KRTAP10-4
BX322557                  BX322557
ENSG00000273554    ENSG00000273554
ENSG00000277400    ENSG00000277400
ENSG00000273496    ENSG00000273496
ENSG00000278198    ENSG00000278198
ENSG00000277630    ENSG00000277630
Name: gene_name, dtype: category
Categories (26161, object): ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', ..., 'ZYG11B', 'ZYX', 'ZZEF1', 'ZZZ3']

In [66]:
dataset.var.index

Index(['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'C1ORF112', 'FGR', 'CFH', 'FUCA2',
       'GCLC', 'NFYA',
       ...
       'ELOA3D', 'ELOA3', 'CDR1', 'ACTL10', 'PANO1', 'PRRC2B', 'UGT1A3',
       'UGT1A5', 'F8A2', 'F8A1'],
      dtype='object', length=36517)

In [33]:
geneset_df = pd.read_csv('/home/ddz5/Desktop/c2s-RL/gene_programs_dev/data/msigdb.tsv',sep='\t')

In [65]:
geneset_df

,gene_set,cluster_500,cluster_475,cluster_450,cluster_425,cluster_400,cluster_375,cluster_350,cluster_325,cluster_300,...,cluster_175,cluster_150,cluster_125,cluster_100,cluster_75,cluster_50,cluster_25,cluster_10,cluster_1,genes
0,MT,945,987,1035,11696,111,129,165,133,195,...,283,283,284,287,291,309,214,119,16,"MT-ATP6,MT-ATP8,MT-CO1,MT-CO2,MT-CO3,MT-CYB,MT..."
1,chr10p11,946,988,1036,1075,1124,1167,1164,1145,1118,...,23,23,26,28,35,61,131,169,22,"ABCD1P2,ACTR3BP5,AK3P5,ANKRD30A,ARHGAP12,ARL6I..."
2,chr10p12,947,989,1037,10614,5205,4418,577,478,403,...,144,144,145,147,154,175,226,247,2,"ABI1,ACBD5,ADIPOR1P1,AIFM1P1,AMD1P1,ANKRD26,AP..."
3,chr10p13,948,990,606,671,697,712,714,715,717,...,717,716,720,712,705,676,572,68,0,"ACBD7,BEND7,BEND7-DT,BTBD7P1,C1QL3,CAMK1D,CCDC..."
4,chr10p14,949,991,1038,1076,1125,1168,1922,3823,439,...,449,449,450,455,457,468,402,209,0,"ATP5F1C,CDC123,CELF2,CELF2-AS1,CELF2-AS2,CELF2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34545,HALLMARK_UNFOLDED_PROTEIN_RESPONSE,32,43,50,138,203,234,292,312,350,...,373,374,377,380,386,402,408,79,9,"ALDH18A1,ARFGAP1,ASNS,ATF3,ATF4,ATF6,ATP6V0D1,..."
34546,HALLMARK_UV_RESPONSE_DN,705,613,443,194,178,212,187,258,204,...,227,217,220,223,100,258,8,53,4,"ABCC1,ACVR2A,ADD3,ADGRL2,ADORA2B,AGGF1,AKT3,AM..."
34547,HALLMARK_UV_RESPONSE_UP,0,2,1,10,45,113,179,256,295,...,339,339,340,344,348,359,372,112,9,"ABCB1,ACAA1,AGO2,ALAS1,ALDOA,AMD1,AP2S1,APOM,A..."
34548,HALLMARK_WNT_BETA_CATENIN_SIGNALING,645,627,544,489,406,307,202,200,193,...,245,245,246,251,257,272,302,275,1,"ADAM17,AXIN1,AXIN2,CCND2,CSNK1E,CTNNB1,CUL1,DK..."


In [39]:
genes = set()
for geneset in geneset_df['genes'].values:
    for gene in geneset.split(','):
        genes.add(gene)

In [50]:
len(genes)

42416

In [23]:
msigdb_path = '/home/ddz5/Desktop/c2s-RL/gene_programs_dev/data/msigdb.tsv'

msigdb = pd.read_csv(msigdb_path, sep='\t')

In [24]:
# Recall this contains the ~34.5k gene sets with cluster label at varying resolution levels
msigdb

,gene_set,cluster_500,cluster_475,cluster_450,cluster_425,cluster_400,cluster_375,cluster_350,cluster_325,cluster_300,...,cluster_175,cluster_150,cluster_125,cluster_100,cluster_75,cluster_50,cluster_25,cluster_10,cluster_1,genes
0,MT,945,987,1035,11696,111,129,165,133,195,...,283,283,284,287,291,309,214,119,16,"MT-ATP6,MT-ATP8,MT-CO1,MT-CO2,MT-CO3,MT-CYB,MT..."
1,chr10p11,946,988,1036,1075,1124,1167,1164,1145,1118,...,23,23,26,28,35,61,131,169,22,"ABCD1P2,ACTR3BP5,AK3P5,ANKRD30A,ARHGAP12,ARL6I..."
2,chr10p12,947,989,1037,10614,5205,4418,577,478,403,...,144,144,145,147,154,175,226,247,2,"ABI1,ACBD5,ADIPOR1P1,AIFM1P1,AMD1P1,ANKRD26,AP..."
3,chr10p13,948,990,606,671,697,712,714,715,717,...,717,716,720,712,705,676,572,68,0,"ACBD7,BEND7,BEND7-DT,BTBD7P1,C1QL3,CAMK1D,CCDC..."
4,chr10p14,949,991,1038,1076,1125,1168,1922,3823,439,...,449,449,450,455,457,468,402,209,0,"ATP5F1C,CDC123,CELF2,CELF2-AS1,CELF2-AS2,CELF2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34545,HALLMARK_UNFOLDED_PROTEIN_RESPONSE,32,43,50,138,203,234,292,312,350,...,373,374,377,380,386,402,408,79,9,"ALDH18A1,ARFGAP1,ASNS,ATF3,ATF4,ATF6,ATP6V0D1,..."
34546,HALLMARK_UV_RESPONSE_DN,705,613,443,194,178,212,187,258,204,...,227,217,220,223,100,258,8,53,4,"ABCC1,ACVR2A,ADD3,ADGRL2,ADORA2B,AGGF1,AKT3,AM..."
34547,HALLMARK_UV_RESPONSE_UP,0,2,1,10,45,113,179,256,295,...,339,339,340,344,348,359,372,112,9,"ABCB1,ACAA1,AGO2,ALAS1,ALDOA,AMD1,AP2S1,APOM,A..."
34548,HALLMARK_WNT_BETA_CATENIN_SIGNALING,645,627,544,489,406,307,202,200,193,...,245,245,246,251,257,272,302,275,1,"ADAM17,AXIN1,AXIN2,CCND2,CSNK1E,CTNNB1,CUL1,DK..."


In [67]:
# Load in preprocessed training inputs for c2gsea
import pickle

with open('/home/ddz5/scratch/Cell2GSEA_QA_dataset_models/local_23/training_inputs.pickle', 'rb') as f:
    training_inputs = pickle.load(f)

In [71]:
training_inputs.keys()

dict_keys(['input_gene_set_csv_path', 'input_h5ad_file_path', 'gene_set_names', 'gene_set_sizes', 'gene_names', 'cell_ids', 'extra', 'train_x_sparse', 'train_edges', 'progs_sparse', 'prog_clusters'])